In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import pickle
import time
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    StratifiedKFold,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

print("✓ All libraries imported successfully")

In [ ]:
# Load and Preprocess Data
print("🔄 Loading dataset...")
students_df = pd.read_csv("../datasets/dataset.csv")

# Remove outliers using IQR method
Q1 = students_df.select_dtypes(include=[np.number]).quantile(0.25)
Q3 = students_df.select_dtypes(include=[np.number]).quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = (
    (students_df.select_dtypes(include=[np.number]) < lower_bound)
    | (students_df.select_dtypes(include=[np.number]) > upper_bound)
).any(axis=1)
students_df_cleaned = students_df[~outliers].copy()

# Normalize numerical features
scaler = StandardScaler()
numerical_cols = students_df_cleaned.select_dtypes(include=[np.number]).columns.tolist()
students_df_cleaned[numerical_cols] = scaler.fit_transform(
    students_df_cleaned[numerical_cols]
)

students_df_normalised_no_outliers = students_df_cleaned.copy()

print(f"✓ Data preprocessed: {len(students_df_normalised_no_outliers)} samples")

In [ ]:
# Prepare Features and Target
print("📊 Preparing features and target...")

# Convert Target to numerical
students_df_normalised_no_outliers["Target"] = students_df_normalised_no_outliers[
    "Target"
].replace({"Dropout": 0, "Graduate": 1, "Enrolled": 2})

# Remove enrolled students
students_df_normalised_no_outliers = students_df_normalised_no_outliers[
    students_df_normalised_no_outliers["Target"] != 2
]

# Remove features based on notebook analysis
features_to_remove = [
    "Curricular units 1st sem (credited)",
    "Curricular units 1st sem (enrolled)",
    "Curricular units 1st sem (evaluations)",
    "Curricular units 1st sem (approved)",
    "Curricular units 1st sem (grade)",
    "Curricular units 2nd sem (approved)",
    "Nationality",
]

students_df_normalised_no_outliers = students_df_normalised_no_outliers.drop(
    features_to_remove, axis=1, errors="ignore"
)

# Prepare X and y
X = students_df_normalised_no_outliers.drop("Target", axis=1)
y = students_df_normalised_no_outliers["Target"]

print(f"✓ Features: {X.shape[1]}")
print(f"✓ Samples: {X.shape[0]}")
print(f"✓ Class distribution: {dict(y.value_counts())}")

In [ ]:
# Apply Random Oversampling
print("🔄 Applying Random Oversampling...")
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)
print(f"✓ Resampled: {X_resampled.shape[0]} samples")
print(f"✓ Balanced classes: {dict(pd.Series(y_resampled).value_counts())}")

# Cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store results
results_comparison = []

## 1. Random Forest Tuning


In [15]:
print("🌲" * 40)
print("1. RANDOM FOREST CLASSIFIER - ADVANCED TUNING")
print("🌲" * 40)

# Default performance
print("\n📌 Default Parameters Performance:")
rf_default = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_default_start = time.time()
rf_default_scores = cross_val_score(
    rf_default, X_resampled, y_resampled, cv=cv, scoring="accuracy"
)
rf_default_time = time.time() - rf_default_start
rf_default_mean = rf_default_scores.mean()

print(f"   Accuracy: {rf_default_mean:.4f} (+/- {rf_default_scores.std():.4f})")
print(f"   Training Time: {rf_default_time:.2f}s")

# Advanced Tuning - Expanded Grid
print("\n🔧 Advanced Hyperparameter Tuning...")
rf_param_grid = {
    "n_estimators": [100, 200, 300, 400],
    "max_depth": [15, 20, 25, 30, None],
    "min_samples_split": [2, 3, 5, 7],
    "min_samples_leaf": [1, 2, 3, 4],
    "max_features": ["sqrt", "log2", 0.5, 0.7],
    "criterion": ["gini", "entropy"],
    "bootstrap": [True, False],
}

print(f"   Total combinations: {4*5*4*4*4*2*2} = 5,120")

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_param_grid,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2,
)

rf_tuning_start = time.time()
rf_grid.fit(X_resampled, y_resampled)
rf_tuning_time = time.time() - rf_tuning_start

print(f"\n✓ Tuning completed in {rf_tuning_time:.2f}s")
print(f"✓ Best Parameters: {rf_grid.best_params_}")
print(f"✓ Best CV Score: {rf_grid.best_score_:.4f}")

rf_improvement = ((rf_grid.best_score_ - rf_default_mean) / rf_default_mean) * 100
print(f"📈 Improvement: {rf_improvement:+.2f}%")

results_comparison.append(
    {
        "Model": "Random Forest (Advanced)",
        "Default Accuracy": f"{rf_default_mean:.4f}",
        "Tuned Accuracy": f"{rf_grid.best_score_:.4f}",
        "Improvement": f"{rf_improvement:+.2f}%",
        "Best Params": str(rf_grid.best_params_),
        "Tuning Time": f"{rf_tuning_time:.1f}s",
    }
)


✓ Tuning completed in 7970.00s
✓ Best Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
✓ Best CV Score: 0.9816
📈 Improvement: +1.91%


## 2. Decision Tree Tuning


In [16]:
print("🌳" * 40)
print("2. DECISION TREE CLASSIFIER - ADVANCED TUNING")
print("🌳" * 40)

# Default performance
print("\n📌 Default Parameters Performance:")
dt_default = DecisionTreeClassifier(random_state=42)
dt_default_start = time.time()
dt_default_scores = cross_val_score(
    dt_default, X_resampled, y_resampled, cv=cv, scoring="accuracy"
)
dt_default_time = time.time() - dt_default_start
dt_default_mean = dt_default_scores.mean()

print(f"   Accuracy: {dt_default_mean:.4f} (+/- {dt_default_scores.std():.4f})")
print(f"   Training Time: {dt_default_time:.2f}s")

# Advanced Tuning - Expanded Grid
print("\n🔧 Advanced Hyperparameter Tuning...")
dt_param_grid = {
    "max_depth": [5, 10, 15, 20, 25, 30, 35, None],
    "min_samples_split": [2, 3, 5, 7, 10, 15, 20],
    "min_samples_leaf": [1, 2, 3, 4, 5, 6, 8],
    "criterion": ["gini", "entropy", "log_loss"],
    "splitter": ["best", "random"],
    "max_features": ["sqrt", "log2", None],
    "ccp_alpha": [0.0, 0.001, 0.01, 0.05],
}

print(f"   Total combinations: {8*7*7*3*2*3*4} = 28,224")

dt_grid = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    dt_param_grid,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2,
)

dt_tuning_start = time.time()
dt_grid.fit(X_resampled, y_resampled)
dt_tuning_time = time.time() - dt_tuning_start

print(f"\n✓ Tuning completed in {dt_tuning_time:.2f}s")
print(f"✓ Best Parameters: {dt_grid.best_params_}")
print(f"✓ Best CV Score: {dt_grid.best_score_:.4f}")

dt_improvement = ((dt_grid.best_score_ - dt_default_mean) / dt_default_mean) * 100
print(f"📈 Improvement: {dt_improvement:+.2f}%")

results_comparison.append(
    {
        "Model": "Decision Tree (Advanced)",
        "Default Accuracy": f"{dt_default_mean:.4f}",
        "Tuned Accuracy": f"{dt_grid.best_score_:.4f}",
        "Improvement": f"{dt_improvement:+.2f}%",
        "Best Params": str(dt_grid.best_params_),
        "Tuning Time": f"{dt_tuning_time:.1f}s",
    }
)

🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳
2. DECISION TREE CLASSIFIER - ADVANCED TUNING
🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳🌳

📌 Default Parameters Performance:
   Accuracy: 0.9163 (+/- 0.0142)
   Training Time: 0.17s

🔧 Advanced Hyperparameter Tuning...
   Total combinations: 28224 = 28,224
Fitting 5 folds for each of 28224 candidates, totalling 141120 fits
   Accuracy: 0.9163 (+/- 0.0142)
   Training Time: 0.17s

🔧 Advanced Hyperparameter Tuning...
   Total combinations: 28224 = 28,224
Fitting 5 folds for each of 28224 candidates, totalling 141120 fits

✓ Tuning completed in 413.27s
✓ Best Parameters: {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 15, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'random'}
✓ Best CV Score: 0.9372
📈 Improvement: +2.28%

✓ Tuning completed in 413.27s
✓ Best Parameters: {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 15, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'rand

## 3. Logistic Regression Tuning


In [17]:
print("📊" * 40)
print("3. LOGISTIC REGRESSION - ADVANCED TUNING")
print("📊" * 40)

# Default performance
print("\n📌 Default Parameters Performance:")
lr_default = LogisticRegression(max_iter=1000, random_state=42)
lr_default_start = time.time()
lr_default_scores = cross_val_score(
    lr_default, X_resampled, y_resampled, cv=cv, scoring="accuracy"
)
lr_default_time = time.time() - lr_default_start
lr_default_mean = lr_default_scores.mean()

print(f"   Accuracy: {lr_default_mean:.4f} (+/- {lr_default_scores.std():.4f})")
print(f"   Training Time: {lr_default_time:.2f}s")

# Advanced Tuning - Expanded Grid
print("\n🔧 Advanced Hyperparameter Tuning...")
lr_param_grid = {
    "C": [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100, 500],
    "penalty": ["l1", "l2", "elasticnet", None],
    "solver": ["lbfgs", "liblinear", "saga", "newton-cg"],
    "max_iter": [500, 1000, 1500, 2000, 3000],
    "class_weight": [None, "balanced", {0: 1, 1: 2}, {0: 1, 1: 3}],
    "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
}

lr_random = RandomizedSearchCV(
    LogisticRegression(random_state=42),
    lr_param_grid,
    n_iter=100,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2,
    random_state=42,
)

lr_tuning_start = time.time()
lr_random.fit(X_resampled, y_resampled)
lr_tuning_time = time.time() - lr_tuning_start

print(f"\n✓ Tuning completed in {lr_tuning_time:.2f}s")
print(f"✓ Best Parameters: {lr_random.best_params_}")
print(f"✓ Best CV Score: {lr_random.best_score_:.4f}")

lr_improvement = ((lr_random.best_score_ - lr_default_mean) / lr_default_mean) * 100
print(f"📈 Improvement: {lr_improvement:+.2f}%")

results_comparison.append(
    {
        "Model": "Logistic Regression (Advanced)",
        "Default Accuracy": f"{lr_default_mean:.4f}",
        "Tuned Accuracy": f"{lr_random.best_score_:.4f}",
        "Improvement": f"{lr_improvement:+.2f}%",
        "Best Params": str(lr_random.best_params_),
        "Tuning Time": f"{lr_tuning_time:.1f}s",
    }
)

📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊
3. LOGISTIC REGRESSION - ADVANCED TUNING
📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊

📌 Default Parameters Performance:
   Accuracy: 0.7814 (+/- 0.0192)
   Training Time: 0.13s

🔧 Advanced Hyperparameter Tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits

✓ Tuning completed in 2.32s
✓ Best Parameters: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 500, 'l1_ratio': 0.9, 'class_weight': None, 'C': 1}
✓ Best CV Score: 0.7822
📈 Improvement: +0.11%

✓ Tuning completed in 2.32s
✓ Best Parameters: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 500, 'l1_ratio': 0.9, 'class_weight': None, 'C': 1}
✓ Best CV Score: 0.7822
📈 Improvement: +0.11%


## 4. SVM Tuning


In [18]:
print("⚡" * 40)
print("4. SUPPORT VECTOR MACHINE - ADVANCED TUNING")
print("⚡" * 40)

# Default performance
print("\n📌 Default Parameters Performance:")
svm_default = SVC(random_state=42)
svm_default_start = time.time()
svm_default_scores = cross_val_score(
    svm_default, X_resampled, y_resampled, cv=cv, scoring="accuracy"
)
svm_default_time = time.time() - svm_default_start
svm_default_mean = svm_default_scores.mean()

print(f"   Accuracy: {svm_default_mean:.4f} (+/- {svm_default_scores.std():.4f})")
print(f"   Training Time: {svm_default_time:.2f}s")

# Advanced Tuning - Expanded Grid
print("\n🔧 Advanced Hyperparameter Tuning...")
svm_param_grid = {
    "C": [0.01, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000],
    "gamma": ["scale", "auto", 0.0001, 0.001, 0.01, 0.1, 0.5, 1],
    "kernel": ["rbf", "poly", "sigmoid", "linear"],
    "degree": [2, 3, 4, 5],
    "class_weight": [None, "balanced", {0: 1, 1: 2}, {0: 1, 1: 3}],
    "shrinking": [True, False],
    "cache_size": [500, 1000],
}

svm_random = RandomizedSearchCV(
    SVC(random_state=42),
    svm_param_grid,
    n_iter=100,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2,
    random_state=42,
)

svm_tuning_start = time.time()
svm_random.fit(X_resampled, y_resampled)
svm_tuning_time = time.time() - svm_tuning_start

print(f"\n✓ Tuning completed in {svm_tuning_time:.2f}s")
print(f"✓ Best Parameters: {svm_random.best_params_}")
print(f"✓ Best CV Score: {svm_random.best_score_:.4f}")

svm_improvement = ((svm_random.best_score_ - svm_default_mean) / svm_default_mean) * 100
print(f"📈 Improvement: {svm_improvement:+.2f}%")

results_comparison.append(
    {
        "Model": "SVM (Advanced)",
        "Default Accuracy": f"{svm_default_mean:.4f}",
        "Tuned Accuracy": f"{svm_random.best_score_:.4f}",
        "Improvement": f"{svm_improvement:+.2f}%",
        "Best Params": str(svm_random.best_params_),
        "Tuning Time": f"{svm_tuning_time:.1f}s",
    }
)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
4. SUPPORT VECTOR MACHINE - ADVANCED TUNING
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡

📌 Default Parameters Performance:
   Accuracy: 0.8752 (+/- 0.0210)
   Training Time: 0.51s

🔧 Advanced Hyperparameter Tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
   Accuracy: 0.8752 (+/- 0.0210)
   Training Time: 0.51s

🔧 Advanced Hyperparameter Tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits

✓ Tuning completed in 462.97s
✓ Best Parameters: {'shrinking': False, 'kernel': 'rbf', 'gamma': 1, 'degree': 3, 'class_weight': None, 'cache_size': 500, 'C': 1}
✓ Best CV Score: 0.9950
📈 Improvement: +13.69%

✓ Tuning completed in 462.97s
✓ Best Parameters: {'shrinking': False, 'kernel': 'rbf', 'gamma': 1, 'degree': 3, 'class_weight': None, 'cache_size': 500, 'C': 1}
✓ Best CV Score: 0.9950
📈 Improvement: +13.69%


## 5. KNN Tuning


## 6. Neural Networks Tuning


In [19]:
print("🧠" * 40)
print("6. NEURAL NETWORKS - ADVANCED ARCHITECTURE SEARCH")
print("🧠" * 40)

try:
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    from sklearn.model_selection import train_test_split

    # Split data for neural network
    X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(
        X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
    )

    print("\n📌 Testing advanced neural network architectures...")

    nn_architectures = []

    # Architecture 1: Improved Sigmoid with Heavy Dropout
    print("\n🔧 Architecture 1: Improved Sigmoid + Heavy Dropout + LR Scheduler")
    model1 = Sequential(
        [
            Dense(256, activation="sigmoid", input_shape=(X_train_nn.shape[1],)),
            Dropout(0.4),
            Dense(128, activation="sigmoid"),
            Dropout(0.3),
            Dense(64, activation="sigmoid"),
            Dropout(0.2),
            Dense(32, activation="sigmoid"),
            Dense(1, activation="sigmoid"),
        ]
    )
    model1.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    early_stop = EarlyStopping(
        monitor="val_loss", patience=15, restore_best_weights=True
    )
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5, min_lr=0.00001
    )

    history1 = model1.fit(
        X_train_nn,
        y_train_nn,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop, reduce_lr],
        verbose=0,
    )

    _, acc1 = model1.evaluate(X_test_nn, y_test_nn, verbose=0)
    print(f"   Accuracy: {acc1:.4f}")
    nn_architectures.append(("Sigmoid + Heavy Dropout", acc1, model1))

    # Architecture 2: RELU with BatchNorm + Residual-like
    print("\n🔧 Architecture 2: Deep RELU + BatchNorm + Regularization")
    model2 = Sequential(
        [
            Dense(512, activation="relu", input_shape=(X_train_nn.shape[1],)),
            BatchNormalization(),
            Dropout(0.4),
            Dense(256, activation="relu"),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation="relu"),
            BatchNormalization(),
            Dropout(0.3),
            Dense(64, activation="relu"),
            BatchNormalization(),
            Dropout(0.2),
            Dense(32, activation="relu"),
            Dense(1, activation="sigmoid"),
        ]
    )
    model2.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history2 = model2.fit(
        X_train_nn,
        y_train_nn,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop, reduce_lr],
        verbose=0,
    )

    _, acc2 = model2.evaluate(X_test_nn, y_test_nn, verbose=0)
    print(f"   Accuracy: {acc2:.4f}")
    nn_architectures.append(("Deep RELU + BatchNorm", acc2, model2))

    # Architecture 3: Leaky RELU (fixes dying neurons)
    print("\n🔧 Architecture 3: Leaky RELU + Advanced Regularization")
    model3 = Sequential(
        [
            Dense(384, input_shape=(X_train_nn.shape[1],)),
            LeakyReLU(alpha=0.1),
            BatchNormalization(),
            Dropout(0.4),
            Dense(192),
            LeakyReLU(alpha=0.1),
            BatchNormalization(),
            Dropout(0.3),
            Dense(96),
            LeakyReLU(alpha=0.1),
            BatchNormalization(),
            Dropout(0.2),
            Dense(48),
            LeakyReLU(alpha=0.1),
            Dense(1, activation="sigmoid"),
        ]
    )
    model3.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history3 = model3.fit(
        X_train_nn,
        y_train_nn,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop, reduce_lr],
        verbose=0,
    )

    _, acc3 = model3.evaluate(X_test_nn, y_test_nn, verbose=0)
    print(f"   Accuracy: {acc3:.4f}")
    nn_architectures.append(("Leaky RELU + Advanced", acc3, model3))

    # Architecture 4: Wide & Deep Network
    print("\n🔧 Architecture 4: Wide & Deep Network (512-256-128-64)")
    model4 = Sequential(
        [
            Dense(512, activation="relu", input_shape=(X_train_nn.shape[1],)),
            BatchNormalization(),
            Dropout(0.5),
            Dense(256, activation="relu"),
            BatchNormalization(),
            Dropout(0.4),
            Dense(128, activation="relu"),
            BatchNormalization(),
            Dropout(0.3),
            Dense(64, activation="relu"),
            BatchNormalization(),
            Dropout(0.2),
            Dense(1, activation="sigmoid"),
        ]
    )
    model4.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history4 = model4.fit(
        X_train_nn,
        y_train_nn,
        epochs=100,
        batch_size=64,
        validation_split=0.2,
        callbacks=[early_stop, reduce_lr],
        verbose=0,
    )

    _, acc4 = model4.evaluate(X_test_nn, y_test_nn, verbose=0)
    print(f"   Accuracy: {acc4:.4f}")
    nn_architectures.append(("Wide & Deep", acc4, model4))

    # Architecture 5: Optimized Leaky RELU with Smaller Batch
    print("\n🔧 Architecture 5: Optimized Leaky RELU + Small Batch Training")
    model5 = Sequential(
        [
            Dense(320, input_shape=(X_train_nn.shape[1],)),
            LeakyReLU(alpha=0.2),
            BatchNormalization(),
            Dropout(0.5),
            Dense(160),
            LeakyReLU(alpha=0.2),
            BatchNormalization(),
            Dropout(0.4),
            Dense(80),
            LeakyReLU(alpha=0.2),
            BatchNormalization(),
            Dropout(0.3),
            Dense(40),
            LeakyReLU(alpha=0.2),
            Dense(1, activation="sigmoid"),
        ]
    )
    model5.compile(
        optimizer=Adam(learning_rate=0.0008),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history5 = model5.fit(
        X_train_nn,
        y_train_nn,
        epochs=100,
        batch_size=16,
        validation_split=0.2,
        callbacks=[early_stop, reduce_lr],
        verbose=0,
    )

    _, acc5 = model5.evaluate(X_test_nn, y_test_nn, verbose=0)
    print(f"   Accuracy: {acc5:.4f}")
    nn_architectures.append(("Optimized Leaky RELU", acc5, model5))

    # Find best neural network
    best_nn = max(nn_architectures, key=lambda x: x[1])

    print(f"\n✓ Best Neural Network: {best_nn[0]}")
    print(f"✓ Best Accuracy: {best_nn[1]:.4f}")

    # Compare with default (70% RELU from original)
    nn_default_acc = 0.70
    nn_improvement = ((best_nn[1] - nn_default_acc) / nn_default_acc) * 100

    results_comparison.append(
        {
            "Model": f"Neural Network ({best_nn[0]})",
            "Default Accuracy": f"{nn_default_acc:.4f}",
            "Tuned Accuracy": f"{best_nn[1]:.4f}",
            "Improvement": f"{nn_improvement:+.2f}%",
            "Best Params": f"{best_nn[0]}, BatchNorm, Dropout, EarlyStopping, LR Scheduler",
            "Tuning Time": "N/A (architecture search)",
        }
    )

    print(f"\n📈 Improvement over default RELU: {nn_improvement:+.2f}%")

    # Save best neural network model
    best_model = best_nn[2]
    best_model.save("../trained-models/nn_tuned_advanced.keras")
    print(f"✓ Best neural network saved to: ../trained-models/nn_tuned_advanced.keras")

except ImportError as e:
    print(f"\n⚠️  TensorFlow not available: {e}")
    print("Skipping neural network tuning...")
except Exception as e:
    print(f"\n❌ Error during neural network tuning: {e}")
    import traceback

    traceback.print_exc()

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠
6. NEURAL NETWORKS - ADVANCED ARCHITECTURE SEARCH
🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

📌 Testing advanced neural network architectures...

🔧 Architecture 1: Improved Sigmoid + Heavy Dropout + LR Scheduler

📌 Testing advanced neural network architectures...

🔧 Architecture 1: Improved Sigmoid + Heavy Dropout + LR Scheduler
   Accuracy: 0.7741

🔧 Architecture 2: Deep RELU + BatchNorm + Regularization
   Accuracy: 0.7741

🔧 Architecture 2: Deep RELU + BatchNorm + Regularization
   Accuracy: 0.8787

🔧 Architecture 3: Leaky RELU + Advanced Regularization
   Accuracy: 0.8787

🔧 Architecture 3: Leaky RELU + Advanced Regularization
   Accuracy: 0.7197

🔧 Architecture 4: Wide & Deep Network (512-256-128-64)
   Accuracy: 0.7197

🔧 Architecture 4: Wide & Deep Network (512-256-128-64)
   Accuracy: 0.7448

🔧 Architecture 5: Optimized Leaky RELU + Small Batch Training
   Accuracy: 0.7448

🔧 Architecture 5: Optimized Leaky RELU + Small Batch Training
   A

In [20]:
print("🎯" * 40)
print("5. K-NEAREST NEIGHBORS - ADVANCED TUNING")
print("🎯" * 40)

# Default performance
print("\n📌 Default Parameters Performance:")
knn_default = KNeighborsClassifier(n_neighbors=5)
knn_default_start = time.time()
knn_default_scores = cross_val_score(
    knn_default, X_resampled, y_resampled, cv=cv, scoring="accuracy"
)
knn_default_time = time.time() - knn_default_start
knn_default_mean = knn_default_scores.mean()

print(f"   Accuracy: {knn_default_mean:.4f} (+/- {knn_default_scores.std():.4f})")
print(f"   Training Time: {knn_default_time:.2f}s")

# Advanced Tuning - Expanded Grid
print("\n🔧 Advanced Hyperparameter Tuning...")
knn_param_grid = {
    "n_neighbors": [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 25, 31],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "minkowski", "chebyshev", "hamming"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [10, 20, 30, 40, 50, 60],
    "p": [1, 2, 3, 4],
}

knn_random = RandomizedSearchCV(
    KNeighborsClassifier(),
    knn_param_grid,
    n_iter=100,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2,
    random_state=42,
)

knn_tuning_start = time.time()
knn_random.fit(X_resampled, y_resampled)
knn_tuning_time = time.time() - knn_tuning_start

print(f"\n✓ Tuning completed in {knn_tuning_time:.2f}s")
print(f"✓ Best Parameters: {knn_random.best_params_}")
print(f"✓ Best CV Score: {knn_random.best_score_:.4f}")

knn_improvement = ((knn_random.best_score_ - knn_default_mean) / knn_default_mean) * 100
print(f"📈 Improvement: {knn_improvement:+.2f}%")

results_comparison.append(
    {
        "Model": "KNN (Advanced)",
        "Default Accuracy": f"{knn_default_mean:.4f}",
        "Tuned Accuracy": f"{knn_random.best_score_:.4f}",
        "Improvement": f"{knn_improvement:+.2f}%",
        "Best Params": str(knn_random.best_params_),
        "Tuning Time": f"{knn_tuning_time:.1f}s",
    }
)

🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
5. K-NEAREST NEIGHBORS - ADVANCED TUNING
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯

📌 Default Parameters Performance:
   Accuracy: 0.8174 (+/- 0.0170)
   Training Time: 0.15s

🔧 Advanced Hyperparameter Tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
   Accuracy: 0.8174 (+/- 0.0170)
   Training Time: 0.15s

🔧 Advanced Hyperparameter Tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits

✓ Tuning completed in 5.02s
✓ Best Parameters: {'weights': 'distance', 'p': 4, 'n_neighbors': 3, 'metric': 'hamming', 'leaf_size': 50, 'algorithm': 'ball_tree'}
✓ Best CV Score: 0.9121
📈 Improvement: +11.58%

✓ Tuning completed in 5.02s
✓ Best Parameters: {'weights': 'distance', 'p': 4, 'n_neighbors': 3, 'metric': 'hamming', 'leaf_size': 50, 'algorithm': 'ball_tree'}
✓ Best CV Score: 0.9121
📈 Improvement: +11.58%


## 7. Save Results and Models


In [21]:
# Create results DataFrame
results_df = pd.DataFrame(results_comparison)

print("\n" + "=" * 80)
print("TUNING RESULTS SUMMARY")
print("=" * 80)
print(results_df.to_string(index=False))

# Save results
results_df.to_csv("../trained-models/tuning_results.csv", index=False)
print("\n✓ Results saved to: ../trained-models/tuning_results.csv")

# Save tuned models
with open("../trained-models/rf_tuned_model.pkl", "wb") as f:
    pickle.dump(rf_grid.best_estimator_, f)

with open("../trained-models/dt_tuned_model.pkl", "wb") as f:
    pickle.dump(dt_grid.best_estimator_, f)

with open("../trained-models/lr_tuned_model.pkl", "wb") as f:
    pickle.dump(lr_random.best_estimator_, f)

with open("../trained-models/svm_tuned_model.pkl", "wb") as f:
    pickle.dump(svm_random.best_estimator_, f)

with open("../trained-models/knn_tuned_model.pkl", "wb") as f:
    pickle.dump(knn_random.best_estimator_, f)

print("✓ All tuned models saved successfully!")
print("\n" + "=" * 80)
print("HYPERPARAMETER TUNING COMPLETE!")
print("=" * 80)


TUNING RESULTS SUMMARY
                                 Model Default Accuracy Tuned Accuracy Improvement                                                                                                                                            Best Params               Tuning Time
              Random Forest (Advanced)           0.9632         0.9816      +1.91% {'bootstrap': False, 'criterion': 'gini', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}                   7970.0s
              Decision Tree (Advanced)           0.9163         0.9372      +2.28%    {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 15, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'random'}                    413.3s
        Logistic Regression (Advanced)           0.7814         0.7822      +0.11%                                                    {'solver': 'saga', 'penalty': 'l1', 'max_iter': 500, 'l1_ratio